# Databricks SDK

This page considers details on working with databricks SDK.

## OpenAI client

The method `serving_endpoints.get_open_ai_client.get_open_ai_client` returns the  `openai.OpenAI` client, which you can use to requiest some served models.

---

The following cell creates the `open_ai_client` and shows that it is really open ai client.

In [1]:
from databricks.sdk import WorkspaceClient
w = WorkspaceClient()

open_ai_client = w.serving_endpoints.get_open_ai_client()
type(open_ai_client)

openai.OpenAI

The following cell illustrates the invocation of the embedding model.

In [2]:

embedding = open_ai_client.embeddings.create(
   model="databricks-gte-large-en",
   input="hello"
)
type(embedding)

openai.types.create_embedding_response.CreateEmbeddingResponse

The result is an `openai` embedding response object.

In [3]:
embedding.data[0].embedding[:20]

[-0.9521484375,
 -0.7998046875,
 -0.79931640625,
 -0.138427734375,
 -0.79150390625,
 -0.31787109375,
 -0.55810546875,
 0.392333984375,
 -0.36767578125,
 0.4013671875,
 -0.0791015625,
 -0.78515625,
 -0.4599609375,
 0.4189453125,
 0.418212890625,
 -0.36767578125,
 -0.587890625,
 -0.466796875,
 0.159423828125,
 -0.359130859375]